In [1]:
import pandas as pd
import numpy as np


# Open the dataset

In [2]:
#open the dataset
df = pd.read_csv("household_spending.csv")

/var/folders/kr/gz_9fn415s1dqh3m1jh9rmcc0000gs/T/ipykernel_15544/1814403755.py:2: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("household_spending.csv")


An error occurs due to inconsitent data type in the 14th column, "SYMBOL". Since I do not need to use this column, I simply silenced the warning:


In [3]:
df = pd.read_csv("household_spending.csv", low_memory=False)

In [4]:
df

,REF_DATE,GEO,DGUID,Statistic,"Household expenditures, summary-level categories",UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,2010,Canada,2016A000011124,Average expenditure per household,Total expenditure,Dollars,81,units,0,v54325508,1.1.1,72075.0,NaN,NaN,NaN,0
1,2010,Canada,2016A000011124,Average expenditure per household,Total current consumption,Dollars,81,units,0,v54325509,1.1.2,54013.0,NaN,NaN,NaN,0
2,2010,Canada,2016A000011124,Average expenditure per household,Food expenditures,Dollars,81,units,0,v54325510,1.1.3,7850.0,NaN,NaN,NaN,0
3,2010,Canada,2016A000011124,Average expenditure per household,Food purchased from stores,Dollars,81,units,0,v54325511,1.1.4,5709.0,NaN,NaN,NaN,0
4,2010,Canada,2016A000011124,Average expenditure per household,Bakery products,Dollars,81,units,0,v64469102,1.1.43,581.0,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36512,2019,British Columbia,2016A000259,Average expenditure per household,Gifts of money and support payments,Dollars,81,units,0,v64472646,13.1.311,2837.0,NaN,NaN,NaN,0
36513,2019,British Columbia,2016A000259,Average expenditure per household,Gifts of money to persons living in Canada,Dollars,81,units,0,v64472647,13.1.312,2187.0,NaN,NaN,NaN,0
36514,2019,British Columbia,2016A000259,Average expenditure per household,Gifts of money to persons living outside Canada,Dollars,81,units,0,v64472648,13.1.313,331.0,NaN,NaN,NaN,0
36515,2019,British Columbia,2016A000259,Average expenditure per household,Alimony and child support,Dollars,81,units,0,v64472649,13.1.314,320.0,NaN,NaN,NaN,0


# Data cleaning 

First, let's take a look at all the columns provide in the original dataset.

In [5]:
# list all column names
df.columns

Index(['REF_DATE', 'GEO', 'DGUID', 'Statistic',
       'Household expenditures, summary-level categories', 'UOM', 'UOM_ID',
       'SCALAR_FACTOR', 'SCALAR_ID', 'VECTOR', 'COORDINATE', 'VALUE', 'STATUS',
       'SYMBOL', 'TERMINATED', 'DECIMALS'],
      dtype='object')

Out of these columns, we are only keeping several relevant columns for this project's analysis. They are
* REF_DATE: the year when data was collected
* Household expenditures, summary-level categories: the type of expenses
* VALUE: amount of money that Canadian households spent on the expense in CAD  

Currently, the name of these three columns are vague and not clear. After selecting these three columns, I am going going to rename them.

In [6]:
# drop unwanted columns
columns = ['DGUID', 'Statistic','UOM', 'UOM_ID',
       'SCALAR_FACTOR', 'SCALAR_ID', 'VECTOR', 'COORDINATE', 'STATUS',
       'SYMBOL', 'TERMINATED', 'DECIMALS']
df_1 = df.drop(columns = columns)

In [7]:
# rename columns
df_1.rename(columns = {'REF_DATE':'Year', 'Household expenditures, summary-level categories':'Expense Type', 'VALUE':'Amount' }, inplace = True)

In [8]:
df_1

,Year,GEO,Expense Type,Amount
0,2010,Canada,Total expenditure,72075.0
1,2010,Canada,Total current consumption,54013.0
2,2010,Canada,Food expenditures,7850.0
3,2010,Canada,Food purchased from stores,5709.0
4,2010,Canada,Bakery products,581.0
...,...,...,...,...
36512,2019,British Columbia,Gifts of money and support payments,2837.0
36513,2019,British Columbia,Gifts of money to persons living in Canada,2187.0
36514,2019,British Columbia,Gifts of money to persons living outside Canada,331.0
36515,2019,British Columbia,Alimony and child support,320.0


Look neat? Yes. However, the data is not ready for analysis yet.  
By refering to the metadata file, I know that not all expenses are equal. For example, total current consumption is a subset of total expenditure, while food expenditure is a subset of total current consumption. We need to do further data transformation to correctly classify the type of expenses. For now, let's switch to the metadata file that comes with the dataset.

# Classifying expense types into categories

In [9]:
#read the metadata file
df_metadata = pd.read_csv("11100222_MetaData.csv")

In [10]:
df_metadata.head(10)

,Cube Title,Product Id,CANSIM Id,URL,Cube Notes,Archive Status,Frequency,Start Reference Period,End Reference Period,Total number of dimensions,Unnamed: 10,Unnamed: 11
0,"Household spending, Canada, regions and provinces",11100222,203-0021,https://www150.statcan.gc.ca/t1/tbl1/en/tv.act...,3;4;6;7;11;14,CURRENT - a cube available to the public and t...,Annual,1/1/10,1/1/19,3.0,NaN,Total current consumption
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Income taxes
2,Dimension ID,Dimension name,Dimension Notes,Dimension Definitions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Personal insurance payments and pension contri...
3,1,Geography,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Gifts of money, support payments and charitabl..."
4,2,Statistic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3,"Household expenditures, summary-level categories",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Dimension ID,Member Name,Classification Code,Member ID,Parent Member ID,Terminated,Member Notes,Member Definitions,NaN,NaN,NaN,NaN
8,1,Canada,[11124],1,NaN,NaN,10,NaN,NaN,NaN,NaN,NaN
9,1,Atlantic Region,[1],2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The medata file contains various information about the main dataset. We are going to slice a certain part of the file, from row 7 to row 379, to understand how some expenses are subsets of other expenses.

In [11]:
# select row 7 to row 379, resetting index
df_metadata = df_metadata[7:380].reset_index()
df_metadata.head(5)

,index,Cube Title,Product Id,CANSIM Id,URL,Cube Notes,Archive Status,Frequency,Start Reference Period,End Reference Period,Total number of dimensions,Unnamed: 10,Unnamed: 11
0,7,Dimension ID,Member Name,Classification Code,Member ID,Parent Member ID,Terminated,Member Notes,Member Definitions,NaN,NaN,NaN,NaN
1,8,1,Canada,[11124],1,NaN,NaN,10,NaN,NaN,NaN,NaN,NaN
2,9,1,Atlantic Region,[1],2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10,1,Newfoundland and Labrador,[10],3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,1,Prince Edward Island,[11],4,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# taking columns name from row 0
df_metadata.columns = df_metadata.iloc[0]

In [13]:
# further remove non-expense rows of data
df_metadata = df_metadata[14: ].reset_index()
df_metadata.head(5)

,index,7,Dimension ID,Member Name,Classification Code,Member ID,Parent Member ID,Terminated,Member Notes,Member Definitions,NaN,NaN,NaN,NaN
0,14,21,2,Average expenditure per household,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15,22,3,Total expenditure,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16,23,3,Total current consumption,NaN,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,17,24,3,Food expenditures,NaN,3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18,25,3,Food purchased from stores,NaN,4,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In this metadata dataframe, we are interested in a few columns:
* Member Name: name of the expense
* Member ID: an unique ID assigned to that expense*
* Parent Member ID: if the expense is a subset of another expense, an ID of the parent expense is shown

In [14]:
df_metadata = df_metadata[["Member Name", "Member ID", "Parent Member ID"]]
df_metadata

,Member Name,Member ID,Parent Member ID
0,Average expenditure per household,1,NaN
1,Total expenditure,1,NaN
2,Total current consumption,2,1
3,Food expenditures,3,2
4,Food purchased from stores,4,3
...,...,...,...
354,"Tobacco products, alcoholic beverages and cann...",359,2
355,"Other tobacco products, smokers' supplies and ...",360,279
356,Cannabis for non-medical use,361,37
357,Administration fees for brokers and stock and ...,362,290


We will now create a dictionary that map each expense name to its parent expense name.

In [15]:
# create a dictionary using ID and expense name
expense_name_dict = {}
for i in range(len(df_metadata)):
    id = df_metadata.loc[i, "Member ID"]
    expense_name = df_metadata.loc[i, "Member Name"]
    expense_name_dict[id] = expense_name
expense_name_dict    

{'1': 'Total expenditure',
 '2': 'Total current consumption',
 '3': 'Food expenditures',
 '4': 'Food purchased from stores',
 '5': 'Food purchased from restaurants',
 '6': 'Shelter',
 '7': 'Principal accommodation',
 '8': 'Rented living quarters',
 '9': 'Owned living quarters',
 '10': 'Water, fuel and electricity for principal accommodation',
 '11': 'Other accommodation',
 '12': 'Household operations',
 '13': 'Communications',
 '14': 'Household furnishings and equipment',
 '15': 'Household furnishings',
 '16': 'Household equipment',
 '17': 'Household appliances',
 '18': 'Clothing and accessories',
 '23': 'Transportation',
 '24': 'Private transportation',
 '25': 'Public transportation',
 '26': 'Health care',
 '27': 'Direct health care costs to household',
 '28': 'Health insurance premiums',
 '29': 'Personal care',
 '30': 'Recreation',
 '31': 'Recreational equipment and related services',
 '32': 'Home entertainment equipment and services',
 '33': 'Recreational services',
 '34': 'Recreati

In [16]:
# create a dictionary to link an expense with its parent expense
parent_expense_dict = {}
for i in range(len(df_metadata)):
    try:
        expense_id = df_metadata.loc[i, "Member ID"]
        expense = expense_name_dict[expense_id]
        parent_expense_id = df_metadata.loc[i, "Parent Member ID"]
        parent_expense = expense_name_dict[parent_expense_id]
        parent_expense_dict[expense] = parent_expense
    except:
        print(f'{expense}: no parent expense')
parent_expense_dict

Total expenditure: no parent expense
Total expenditure: no parent expense


{'Total current consumption': 'Total expenditure',
 'Food expenditures': 'Total current consumption',
 'Food purchased from stores': 'Food expenditures',
 'Food purchased from restaurants': 'Food expenditures',
 'Shelter': 'Total current consumption',
 'Principal accommodation': 'Shelter',
 'Rented living quarters': 'Principal accommodation',
 'Owned living quarters': 'Principal accommodation',
 'Water, fuel and electricity for principal accommodation': 'Principal accommodation',
 'Other accommodation': 'Shelter',
 'Household operations': 'Total current consumption',
 'Communications': 'Household operations',
 'Household furnishings and equipment': 'Total current consumption',
 'Household furnishings': 'Household furnishings and equipment',
 'Household equipment': 'Household furnishings and equipment',
 'Household appliances': 'Household equipment',
 'Clothing and accessories': 'Total current consumption',
 'Transportation': 'Total current consumption',
 'Private transportation': 'Tran

For each expense, we are going to add a column showing its parent expense, if any.

In [17]:
df_metadata["Parent Member Name 1"] = ''
for i in range(len(df_metadata)):

    try:
        expense_name = df_metadata.loc[i, "Member Name"]
        parent_name = parent_expense_dict[expense_name]
        df_metadata.loc[i, "Parent Member Name 1"] = parent_name
    except:
        pass
        

In [18]:
df_metadata

,Member Name,Member ID,Parent Member ID,Parent Member Name 1
0,Average expenditure per household,1,NaN,
1,Total expenditure,1,NaN,
2,Total current consumption,2,1,Total expenditure
3,Food expenditures,3,2,Total current consumption
4,Food purchased from stores,4,3,Food expenditures
...,...,...,...,...
354,"Tobacco products, alcoholic beverages and cann...",359,2,Total current consumption
355,"Other tobacco products, smokers' supplies and ...",360,279,Tobacco products and smokers' supplies
356,Cannabis for non-medical use,361,37,Tobacco products and alcoholic beverages
357,Administration fees for brokers and stock and ...,362,290,Financial services


We will repeat the process untill all expenses can be traced back to the widest expense group "Total expenditure"

In [19]:
df_metadata["Parent Member Name 2"] = ''
for i in range(len(df_metadata)):
    try:
        expense_name = df_metadata.loc[i, "Parent Member Name 1"]
        parent_name = parent_expense_dict[expense_name]
        df_metadata.loc[i, "Parent Member Name 2"] = parent_name
    except:
        pass

In [20]:
df_metadata["Parent Member Name 3"] = ''
for i in range(len(df_metadata)):
    try:
        expense_name = df_metadata.loc[i, "Parent Member Name 2"]
        parent_name = parent_expense_dict[expense_name]
        df_metadata.loc[i, "Parent Member Name 3"] = parent_name
    except:
        pass

In [21]:
df_metadata["Parent Member Name 4"] = ''
for i in range(len(df_metadata)):
    try:
        expense_name = df_metadata.loc[i, "Parent Member Name 3"]
        parent_name = parent_expense_dict[expense_name]
        df_metadata.loc[i, "Parent Member Name 4"] = parent_name
    except:
        pass

In [22]:
df_metadata["Parent Member Name 5"] = ''
for i in range(len(df_metadata)):
    try:
        expense_name = df_metadata.loc[i, "Parent Member Name 4"]
        parent_name = parent_expense_dict[expense_name]
        df_metadata.loc[i, "Parent Member Name 5"] = parent_name
    except:
        pass

In [23]:
df_metadata["Parent Member Name 6"] = ''
for i in range(len(df_metadata)):
    try:
        expense_name = df_metadata.loc[i, "Parent Member Name 5"]
        parent_name = parent_expense_dict[expense_name]
        df_metadata.loc[i, "Parent Member Name 6"] = parent_name
    except:
        pass

In [24]:
df_metadata["Parent Member Name 7"] = ''
for i in range(len(df_metadata)):
    try:
        expense_name = df_metadata.loc[i, "Parent Member Name 6"]
        parent_name = parent_expense_dict[expense_name]
        df_metadata.loc[i, "Parent Member Name 7"] = parent_name
    except:
        pass
df_metadata["Parent Member Name 8"] = ''
for i in range(len(df_metadata)):
    try:
        expense_name = df_metadata.loc[i, "Parent Member Name 7"]
        parent_name = parent_expense_dict[expense_name]
        df_metadata.loc[i, "Parent Member Name 8"] = parent_name
    except:
        pass
df_metadata.head(10)

,Member Name,Member ID,Parent Member ID,Parent Member Name 1,Parent Member Name 2,Parent Member Name 3,Parent Member Name 4,Parent Member Name 5,Parent Member Name 6,Parent Member Name 7,Parent Member Name 8
0,Average expenditure per household,1,NaN,,,,,,,,
1,Total expenditure,1,NaN,,,,,,,,
2,Total current consumption,2,1,Total expenditure,,,,,,,
3,Food expenditures,3,2,Total current consumption,Total expenditure,,,,,,
4,Food purchased from stores,4,3,Food expenditures,Total current consumption,Total expenditure,,,,,
5,Food purchased from restaurants,5,3,Food expenditures,Total current consumption,Total expenditure,,,,,
6,Shelter,6,2,Total current consumption,Total expenditure,,,,,,
7,Principal accommodation,7,6,Shelter,Total current consumption,Total expenditure,,,,,
8,Rented living quarters,8,7,Principal accommodation,Shelter,Total current consumption,Total expenditure,,,,
9,Owned living quarters,9,7,Principal accommodation,Shelter,Total current consumption,Total expenditure,,,,


In [25]:
for i in range(len(df_metadata)):
    if not pd.isnull(df_metadata.loc[i,"Parent Member Name 8"]):
        print(df_metadata.loc[i,"Parent Member Name 8"])

Column "Parent Member Name 8" is empty, indicating that all expenses in column 7 do not have any parent expense and we have traced all expenses to the biggest parent expense group. 
We can now drop the Member ID and Parent ID columns because we no longer need them.

In [26]:
columns = ["Member Name", "Parent Member Name 1", 
           "Parent Member Name 2", "Parent Member Name 3", 
           "Parent Member Name 4", "Parent Member Name 5", 
           "Parent Member Name 6", "Parent Member Name 7", 
           "Parent Member Name 8"]
df_meta = df_metadata[columns]
df_meta.head(10)

,Member Name,Parent Member Name 1,Parent Member Name 2,Parent Member Name 3,Parent Member Name 4,Parent Member Name 5,Parent Member Name 6,Parent Member Name 7,Parent Member Name 8
0,Average expenditure per household,,,,,,,,
1,Total expenditure,,,,,,,,
2,Total current consumption,Total expenditure,,,,,,,
3,Food expenditures,Total current consumption,Total expenditure,,,,,,
4,Food purchased from stores,Food expenditures,Total current consumption,Total expenditure,,,,,
5,Food purchased from restaurants,Food expenditures,Total current consumption,Total expenditure,,,,,
6,Shelter,Total current consumption,Total expenditure,,,,,,
7,Principal accommodation,Shelter,Total current consumption,Total expenditure,,,,,
8,Rented living quarters,Principal accommodation,Shelter,Total current consumption,Total expenditure,,,,
9,Owned living quarters,Principal accommodation,Shelter,Total current consumption,Total expenditure,,,,


For ease of future analysis, we will create a df_category dataframe, which rewrites from df_meta. df_category shows expense categories "backwards", starting from total expenditure, then smaller expense group, until we reach the specific type of expense.

In [27]:
col_names = ["Expense Type",
            "Category", "Sub-category 1",
            "Sub-category 2", "Sub-category 3",
            "Sub-category 4", "Sub-category 5",
            "Sub-category 6", "Sub-category 7",
            "Sub-category 8"]
df_category = pd.DataFrame(columns = col_names)
df_category.head(5)

,Expense Type,Category,Sub-category 1,Sub-category 2,Sub-category 3,Sub-category 4,Sub-category 5,Sub-category 6,Sub-category 7,Sub-category 8


In [28]:
for i in range(len(df_metadata)):
    df_category.loc[i, "Expense Type"] = df_meta.loc[i, "Member Name"]
    
df_category.head(5)

,Expense Type,Category,Sub-category 1,Sub-category 2,Sub-category 3,Sub-category 4,Sub-category 5,Sub-category 6,Sub-category 7,Sub-category 8
0,Average expenditure per household,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Total expenditure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Total current consumption,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Food expenditures,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Food purchased from stores,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
for i in range(len(df_category)):
    
    for j in range(1, 9):
        col = "Parent Member Name " + str(9-j)

        if df_meta.loc[i, col] != "":
            for k in range(1,9-j+1):
                col = "Parent Member Name " + str(9-j-k+1)
                category = "Sub-category " + str(k)
                df_category.loc[i,category] = df_metadata.loc[i, col]
                
            break
        
df_category.head(15)

,Expense Type,Category,Sub-category 1,Sub-category 2,Sub-category 3,Sub-category 4,Sub-category 5,Sub-category 6,Sub-category 7,Sub-category 8
0,Average expenditure per household,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Total expenditure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Total current consumption,NaN,Total expenditure,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Food expenditures,NaN,Total expenditure,Total current consumption,NaN,NaN,NaN,NaN,NaN,NaN
4,Food purchased from stores,NaN,Total expenditure,Total current consumption,Food expenditures,NaN,NaN,NaN,NaN,NaN
5,Food purchased from restaurants,NaN,Total expenditure,Total current consumption,Food expenditures,NaN,NaN,NaN,NaN,NaN
6,Shelter,NaN,Total expenditure,Total current consumption,NaN,NaN,NaN,NaN,NaN,NaN
7,Principal accommodation,NaN,Total expenditure,Total current consumption,Shelter,NaN,NaN,NaN,NaN,NaN
8,Rented living quarters,NaN,Total expenditure,Total current consumption,Shelter,Principal accommodation,NaN,NaN,NaN,NaN
9,Owned living quarters,NaN,Total expenditure,Total current consumption,Shelter,Principal accommodation,NaN,NaN,NaN,NaN


In [30]:
# checking that Sub-category 8 is empty
for i in range(len(df_category)):
    if not pd.isnull(df_category.loc[i,"Sub-category 8"]):
        print(df_category.loc[i,"Sub-category 8"])

In [31]:
# drop column Sub-category 8 because it is empty
columns = ["Category", "Sub-category 8"]
df_category = df_category.drop(columns = columns)
df_category

,Expense Type,Sub-category 1,Sub-category 2,Sub-category 3,Sub-category 4,Sub-category 5,Sub-category 6,Sub-category 7
0,Average expenditure per household,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Total expenditure,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Total current consumption,Total expenditure,NaN,NaN,NaN,NaN,NaN,NaN
3,Food expenditures,Total expenditure,Total current consumption,NaN,NaN,NaN,NaN,NaN
4,Food purchased from stores,Total expenditure,Total current consumption,Food expenditures,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
354,"Tobacco products, alcoholic beverages and cann...",Total expenditure,Total current consumption,NaN,NaN,NaN,NaN,NaN
355,"Other tobacco products, smokers' supplies and ...",Total expenditure,Total current consumption,Tobacco products and alcoholic beverages,Tobacco products and smokers' supplies,NaN,NaN,NaN
356,Cannabis for non-medical use,Total expenditure,Total current consumption,Tobacco products and alcoholic beverages,NaN,NaN,NaN,NaN
357,Administration fees for brokers and stock and ...,Total expenditure,Total current consumption,Miscellaneous expenditures,Financial services,NaN,NaN,NaN


# Merge two dfs
It is time to go back to our original dataframe, df_1.

In [32]:
df_1

,Year,GEO,Expense Type,Amount
0,2010,Canada,Total expenditure,72075.0
1,2010,Canada,Total current consumption,54013.0
2,2010,Canada,Food expenditures,7850.0
3,2010,Canada,Food purchased from stores,5709.0
4,2010,Canada,Bakery products,581.0
...,...,...,...,...
36512,2019,British Columbia,Gifts of money and support payments,2837.0
36513,2019,British Columbia,Gifts of money to persons living in Canada,2187.0
36514,2019,British Columbia,Gifts of money to persons living outside Canada,331.0
36515,2019,British Columbia,Alimony and child support,320.0


For each row of expense in df_1, we can introduce the categories that the expense is under, by merging df_category into df_1 

In [33]:
combined_df = pd.merge(df_1, df_category, how="left", on="Expense Type")
# combined_df = pd.merge(df_1, df_category, on="Expense Type")

combined_df

,Year,GEO,Expense Type,Amount,Sub-category 1,Sub-category 2,Sub-category 3,Sub-category 4,Sub-category 5,Sub-category 6,Sub-category 7
0,2010,Canada,Total expenditure,72075.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010,Canada,Total current consumption,54013.0,Total expenditure,NaN,NaN,NaN,NaN,NaN,NaN
2,2010,Canada,Food expenditures,7850.0,Total expenditure,Total current consumption,NaN,NaN,NaN,NaN,NaN
3,2010,Canada,Food purchased from stores,5709.0,Total expenditure,Total current consumption,Food expenditures,NaN,NaN,NaN,NaN
4,2010,Canada,Bakery products,581.0,Total expenditure,Total current consumption,Food expenditures,Food purchased from stores,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
36512,2019,British Columbia,Gifts of money and support payments,2837.0,Total expenditure,"Gifts of money, support payments and charitabl...",NaN,NaN,NaN,NaN,NaN
36513,2019,British Columbia,Gifts of money to persons living in Canada,2187.0,Total expenditure,"Gifts of money, support payments and charitabl...",Gifts of money and support payments,NaN,NaN,NaN,NaN
36514,2019,British Columbia,Gifts of money to persons living outside Canada,331.0,Total expenditure,"Gifts of money, support payments and charitabl...",Gifts of money and support payments,NaN,NaN,NaN,NaN
36515,2019,British Columbia,Alimony and child support,320.0,Total expenditure,"Gifts of money, support payments and charitabl...",Gifts of money and support payments,NaN,NaN,NaN,NaN


The cell is now readay for export!

In [34]:
combined_df.to_excel("transformed data.xlsx")